In [49]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
get_data()

In [71]:
# Function to get historical stock data from Yahoo Finance API
def get_stock_data(ticker, start_date):
    data = yf.download(ticker, start=start_date)
    return data

In [72]:
# Function to create sequences for time series prediction
def create_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        seq = data[i : i + sequence_length]
        sequences.append(seq)
    return np.array(sequences)

In [51]:
# Function to build and train the ANN model
def build_ann_model(input_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))
    #model.compile(loss='mean_squared_error', optimizer='adam')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [73]:
stock_ticker = "AAPL"  # Apple Inc. stock ticker
start_date = "2010-01-01"
end_date = "2021-12-31"
sequence_length = 10

# Get stock data
data = get_stock_data(stock_ticker, start_date)

[*********************100%***********************]  1 of 1 completed


In [74]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_close_dime = scaler.fit_transform(data[['Close', 'High']])
scaled_data = MinMaxScaler(feature_range=(0, 1))
scaled_close_data = scaled_data.fit_transform(data[['Close']])

In [75]:
sequences = create_sequences(scaled_close_dime, sequence_length)

In [76]:
# Split data into training and testing sets
train_size = int(len(sequences) * 0.8)
test_size = len(sequences) - train_size
train_data, test_data = sequences[:train_size], sequences[train_size:]

In [77]:
def handleData(data):
    lendf = data.shape[0] -1 
    if data[lendf - 1][0] > data[lendf][0]:
        return 1
    else:
        return 0
    return 0

trained_data_list = list(map(handleData, train_data))
y_train = np.array(trained_data_list)

test_data_list = list(map(handleData, test_data))
y_test = np.array(test_data_list)

In [57]:
# Separate input (X) and output (y) variables
X_train  = train_data[:, :-1]
X_test = test_data[:, :-1]

In [58]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((2724, 9, 2), (681, 9, 2), (2724,), (681,))

In [59]:
y_test = tf.keras.utils.to_categorical(y_test,2)
y_train = tf.keras.utils.to_categorical(y_train,2)

In [60]:
input_dim = X_train.shape[1] * X_train.shape[2] 
input_dim

18

In [61]:
model = build_ann_model(input_dim)
model.fit(X_train.reshape(-1, input_dim), y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
86/86 [==============================] - 1s 2ms/step - loss: 0.6919 - accuracy: 0.5301
Epoch 2/10
86/86 [==============================] - 0s 2ms/step - loss: 0.6914 - accuracy: 0.5316
Epoch 3/10
86/86 [==============================] - 0s 2ms/step - loss: 0.6919 - accuracy: 0.5316
Epoch 4/10
86/86 [==============================] - 0s 2ms/step - loss: 0.6916 - accuracy: 0.5316
Epoch 5/10
86/86 [==============================] - 0s 2ms/step - loss: 0.6913 - accuracy: 0.5316
Epoch 6/10
86/86 [==============================] - 0s 3ms/step - loss: 0.6916 - accuracy: 0.5316
Epoch 7/10
86/86 [==============================] - 0s 2ms/step - loss: 0.6912 - accuracy: 0.5316
Epoch 8/10
86/86 [==============================] - 0s 2ms/step - loss: 0.6911 - accuracy: 0.5316
Epoch 9/10
86/86 [==============================] - 0s 2ms/step - loss: 0.6912 - accuracy: 0.5316
Epoch 10/10
86/86 [==============================] - 0s 3ms/step - loss: 0.6913 - accuracy: 0.5316


In [62]:
#Evaluate the model
train_loss = model.evaluate(X_train.reshape(-1, input_dim), y_train, verbose=0)
#test_loss = model.evaluate(X_test.reshape(-1, input_dim), y_test, verbose=0)
print("Train Loss:", train_loss)
#print("Test Loss:", test_loss)

Train Loss: [0.6908527612686157, 0.5315712094306946]


In [67]:
# Make predictions
train_predict = model.predict(X_train.reshape(-1, input_dim))
test_predict = model.predict(X_test.reshape(-1, input_dim))

22/22 [==============================] - 0s 1ms/step


In [68]:
train_predict

array([[0.53924704, 0.46297994],
       [0.5393448 , 0.462927  ],
       [0.53931046, 0.4629337 ],
       ...,
       [0.5551101 , 0.44751638],
       [0.5564118 , 0.44640124],
       [0.55708665, 0.44555962]], dtype=float32)

In [69]:
test_predict

array([[0.55873764, 0.44488272],
       [0.55901176, 0.44464418],
       [0.5583258 , 0.4454103 ],
       ...,
       [0.58531564, 0.4215148 ],
       [0.5848371 , 0.42187127],
       [0.58487785, 0.42196938]], dtype=float32)

In [70]:
result_df = pd.DataFrame({ "real" : y_test , "pred" :  predication})
result_df

NameError: name 'predication' is not defined

In [31]:
test_predict.shape,y_test.shape

((681, 1), (681,))

In [ ]:
last_day = X_test[-1]


test_predict = model.predict(X_test.reshape(-1, input_dim))

#print(test_predict)
last_predict = scaled_data.inverse_transform(test_predict)
#print(last_predict)

In [ ]:
data.tail()